# Population Height

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import math
import pymc3 as pm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Example data

Assume that we have some height measurements from some population (say students at the University of Oxford). 

In [ ]:
male_heights = np.array([
    194., 179., 175., 185., 183., 182., 190., 181., 183., 179., 191.,
    190., 184., 180., 169., 181., 185., 186., 195., 191., 183., 191.,
    188., 190., 177., 191., 183., 178., 181., 182., 180., 170., 176.,
    177., 186., 193., 195., 164., 178., 178., 182., 176., 196., 191.,
    178., 162., 188., 191., 181., 178., 168., 183., 171., 171., 189.,
    176., 180., 189., 175., 172., 186., 196., 187., 180., 192., 178.,
    196., 185., 191., 197., 182., 190., 169., 197., 171., 177., 174.,
    181., 182., 190., 184., 164., 174., 187., 177., 180., 177., 184.,
    180., 197., 177., 166., 176., 169., 190., 174., 178., 180., 183.,
    184.
])

female_heights = np.array([
    162., 174., 167., 167., 159., 172., 168., 155., 165., 165., 160.,
    165., 163., 151., 158., 155., 156., 169., 175., 170., 151., 162.,
    164., 177., 154., 170., 172., 162., 171., 161., 148., 159., 159.,
    153., 165., 174., 165., 164., 169., 157., 172., 161., 174., 162.,
    163., 167., 162., 165., 156., 151., 169., 171., 164., 182., 164.,
    167., 161., 177., 177., 167., 163., 144., 170., 150., 170., 157.,
    163., 169., 166., 168., 175., 154., 153., 143., 161., 170., 189.,
    163., 174., 163., 157., 167., 167., 152., 167., 155., 147., 163.,
    166., 160., 163., 168., 163., 178., 161., 165., 154., 152., 161.,
    156.
])

### Statistics of our sample

We might calculate some statistics from the data and plot it to examine it in more detail. For example, the mean, $\mu$, and the variance, $\sigma^2$, and hence the standard deviation, $\sigma$, are given by:

\begin{align*}
\mu & = \frac{1}{N} \sum_{i = 1}^{N} x_i \\
\sigma^2 & = \frac{1}{N} \sum_{i = 1}^{N} x_i^2 - \left(\frac{1}{N} \sum_{i = 1}^{N} x_i \right)^2\\
& = \frac{1}{N} \sum_{i = 1}^{N} \left( x_i - \mu \right)^2
\end{align*}

In [ ]:
mean = np.round(np.mean(male_heights), 0)

standard_deviation = np.round(np.std(male_heights), 0)

print('Male: {:.0f} +/- {:.0f}'.format(mean, standard_deviation))

In [ ]:
mean = np.round(np.mean(female_heights), 0)

standard_deviation = np.round(np.std(female_heights), 0)

print('Female: {:.0f} +/- {:.0f}'.format(mean, standard_deviation))

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(male_heights, bins = 70, color="#7A68A6", density=True)
plt.hist(female_heights, bins = 70, color="#467821", density=True)
plt.xlim(140, 210);
plt.xlabel('Height (cm)')

plt.tight_layout();

### Population estimates

When we make measurement of the mean, variance and standard deviation of a sample from a larger population, we typically want to say something about the larger population from which the samples were drawn (i.e. what is the average height of male members of population).

When we do so, we are imagining that there is some distribution from which our sample was drawn, and we are trying to estimate the parameters of that distribution.

To do so, we use statistical estimators. In the case of the mean and variance, these turn out to be very similar to what we calculated above. Now, the mean, $\mu$, and variance, $\sigma^2$, represent the mean and variance of the whole population, and we have estimates of their value, derived from our sample.

\begin{align*}
\hat{\mu} & = \frac{1}{N} \sum_{i = 1}^{N} x_i \\
\hat{\sigma}^2 & = \frac{1}{N-1} \sum_{i = 1}^{N} \left( x_i - \hat{\mu} \right)^2
\end{align*}

Comparing these to the sample mean and variance above we can see that:

\begin{align*}
\hat{\mu} & = \mu \\
\hat{\sigma}^2 & = \frac{N}{N-1} \sigma^2
\end{align*}

Thus the best estimate of the population mean is the mean of the samples, but the best estimate of the population variance is a little bit larger than the sample variance (with the correction size depending on the size of the sample).

### Probabilistic modelling

The estimators used above are often hard to remember and we need to look them up each time we use them. They also limit us to certain settings for which estimators have already been derived. In this course, we will use a powerful probabilistic programmign approach that avoids these issues. We will explicitly describe the probabilistic model that describes the data, and then use automated approaches to estimate the parameters of this model. We call this a 'generative model' because it describes the process by which the data was genera†ed.

We will assume that the samples that we observe are drawn from a normal distribution with some unknown mean and variance:

\begin{equation*}
x_i \sim \mathcal{N}\left(\mu, \sigma^2\right)
\end{equation*}

We can then write down this model as a probabilistic program that describes our observations with 'prior' distributions for the unknown model parameters.

In [ ]:
model = pm.Model() 
    
with model:
    
    sd = pm.Uniform('sd', 0, 20)
        
    mu = pm.Uniform("mu", 140, 190)
    
    observation = pm.Normal("obs", mu=mu, sd=sd, observed=female_heights)

In [ ]:
with model:
    
    trace = pm.sample(5000);

This process generates a 'trace'. We can plot a histogram of this trace to see the 'posterior' distributions of the unknown model parameters. During the course well understand more about this process and exactly what the 'prior' and 'posterior' mean. 

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 5))

axes[0].hist(trace['mu'], histtype='stepfilled', bins=70, alpha=0.85, color="#467821", density=True)
axes[0].set_xlim(140, 210)
axes[0].set_xlabel('Height (cm)')
axes[0].set_title('Female mean height posterior probability density')

axes[1].hist(trace['sd'], histtype='stepfilled', bins=30, alpha=0.85, color="#A60628", density=True)
axes[1].set_xlim(6, 12)
axes[1].set_xlabel('Height (cm)')
axes[1].set_title('Standard deviation posterior probability')

plt.tight_layout()

Note that the results match the estimators above but we also have a clear indication of the uncertainty of these estimates.

We can do the same with the male height measurements.

In [ ]:
model = pm.Model() 
    
with model:
    
    sd = pm.Uniform('sd', 0, 20)
        
    mu = pm.Uniform("mu", 140, 190)
    
    observation = pm.Normal("obs", mu=mu, sd=sd, observed=male_heights)
    
    trace = pm.sample(5000);
    
fig, axes = plt.subplots(2, 1, figsize=(10, 5))

axes[0].hist(trace['mu'], histtype='stepfilled', bins=70, alpha=0.85, color="#7A68A6", density=True)
axes[0].set_xlim(140, 210)
axes[0].set_xlabel('Height (cm)')
axes[0].set_title('Male mean height posterior probability density')

axes[1].hist(trace['sd'], histtype='stepfilled', bins=30, alpha=0.85, color="#A60628", density=True)
axes[1].set_xlim(6, 12)
axes[1].set_xlabel('Height (cm)')
axes[1].set_title('Standard deviation posterior probability')

plt.tight_layout()   

### More complex models

The above example is very simple and doesn't show anything that we couldn't do before. However, what happens if we have lost the original labels of which height measurements belong to male and female members of the population.

In [ ]:
all_heights = np.concatenate([male_heights, female_heights])

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(all_heights, bins = 70, color="#A60628", density=True)
plt.xlim(140, 210);
plt.xlabel('Height (cm)')

plt.tight_layout();

Now the task is impossible with our standard statistical approached. However, we can still build a probabilistic model which explains this data. We now assume there is a 'latent' variable which we do not know - whether the sample is male or female. We ascribe some prior probability to this - 50/50 - and build a model where we encode our prior understanding about the population (in this case, that male members of the population are generally taller than females).

In this case our model is more complex:

\begin{align*}
s_i & \in \{\text{male}, \text{female} \} \\
x_i & \sim \begin{cases} \mathcal{N}\left(\mu, \sigma^2\right) & \text{if } s_i = \text{female} \\
\mathcal{N}\left(\mu + \text{offset}, \sigma^2\right) & \text{otherwise} \\
\end{cases}
\end{align*}

However, we can again write it as a probabilistic program.

In [ ]:
N = len(all_heights)

model = pm.Model() 
    
with model:
    
    sd = pm.Uniform('sd', 0, 20)
        
    mu = pm.Uniform("mu", 140, 190)

    male_or_female = pm.Bernoulli('male_or_female', 0.5, shape=N)
       
    offset = pm.Uniform("offset", 0, 20)

    height = pm.math.switch(male_or_female, mu, mu + offset)
    
    observation = pm.Normal("obs", mu=height, sd=sd, observed=all_heights)
    
    trace = pm.sample(5000);

In [ ]:
female_samples = trace['mu']
male_samples = female_samples + trace['offset']
sd_samples = trace['sd']

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 5))

axes[0].hist(female_samples, histtype='stepfilled', bins=70, alpha=0.85, color="#467821", density=True)
axes[0].set_xlim(140, 210)
axes[0].set_xlabel('Height (cm)')
axes[0].set_title('Female mean height posterior probability density')

axes[1].hist(male_samples, histtype='stepfilled', bins=70, alpha=0.85, color="#7A68A6", density=True)
axes[1].set_xlim(140, 210)
axes[1].set_xlabel('Height (cm)')
axes[1].set_title('Male mean height posterior probability density')

axes[2].hist(sd_samples, histtype='stepfilled', bins=30, alpha=0.85, color="#A60628", density=True)
axes[2].set_xlabel('Height (cm)')
axes[2].set_title('Standard deviation posterior probability')

plt.tight_layout()

The model and the trace give us some more capabilities. We have a latent variable for the sex of each individual sample. We can can investigate the posterior probability of this latent variable.

In [ ]:
male_or_female = np.mean(trace['male_or_female'], 0)

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(np.arange(N), male_or_female, color="#5DA5DA")
plt.xlabel("Population Member")
plt.ylabel("Probability")
plt.title("Posterior probability of being Male or Female")
plt.xlim(1, N);

plt.tight_layout()